In [1]:
from econ_regime.econ_regime import business_cycle, asset_indicators
from price.macro_info import get_econ_info
from strategy.factors.beta import *
from backtest.metric import *

import pandas as pd
import numpy as np
import statsmodels.api as sm
import bs4 as bs
import requests
import datetime as dt
import pickle5 as pickle

In [2]:
equity_df = pd.read_csv('../quant/equity_universe.csv', index_col=0)
equity_df.index = pd.to_datetime(equity_df.index)

In [86]:
equity_universe = equity_df.loc['2011':,].dropna(axis=1)
equity_universe

,A,AAL,AAP,AAPL,ABC,ABT,ACGL,ACN,ADBE,ADI,...,WTW,WY,WYNN,XEL,XOM,XRAY,YUM,ZBH,ZBRA,ZION
date_time,,,,,,,,,,,,,,,,,,,,,
2011-01-03,27.163610,10.040195,60.552662,10.033263,28.549282,17.639103,9.832222,38.638706,31.290001,28.432861,...,73.060722,12.822729,77.632805,15.788007,46.502182,32.227661,28.031958,48.627453,38.200001,21.050661
2011-01-04,26.910648,10.011914,57.538910,10.085626,28.233181,17.805092,9.771111,38.384243,31.510000,28.110697,...,71.834747,12.686455,80.054596,15.855021,46.720490,31.869568,27.603678,47.981678,37.840000,20.707764
2011-01-05,26.852272,10.464432,57.612873,10.168124,28.665747,17.805092,9.785556,38.392178,32.220001,28.170630,...,72.832153,13.082301,81.087448,15.761208,46.595737,32.172558,27.735012,48.017067,37.799999,20.824848
2011-01-06,26.904169,10.596415,55.717705,10.159907,28.998491,17.768208,9.653333,38.535332,32.270000,28.327969,...,73.039963,12.991453,81.678627,15.747799,46.895157,31.988941,27.929159,46.530907,37.480000,20.766296
2011-01-07,26.994974,10.747252,57.206089,10.232668,29.056715,17.841972,9.580000,38.598942,32.040001,28.163143,...,72.915298,13.348357,84.570541,15.888525,47.150887,31.860401,28.277491,46.522053,37.599998,20.548853
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-23,156.562347,18.260000,224.063019,175.262802,127.924423,136.246506,43.480000,396.261078,569.619995,169.451675,...,232.187408,36.561798,88.879997,64.804298,58.691692,54.536747,132.787628,122.040504,582.409973,60.688782
2021-12-27,157.494980,18.170000,228.281143,179.289444,130.265030,138.498337,43.930000,408.070984,577.679993,172.209473,...,234.740433,37.526936,87.580002,65.008598,59.528496,55.048363,135.407288,123.061127,606.330017,61.321358
2021-12-28,157.931503,18.540001,229.854492,178.255432,130.719345,137.529068,44.270000,408.011993,569.359985,171.139740,...,234.406555,37.716183,86.459999,65.786911,59.336128,55.127079,135.377838,123.446266,597.320007,61.418682


In [94]:
def daily_str(df, start, end):
    df = df.loc[start:end,]
    df = df.dropna(axis=1)
    daily_volatility = df.pct_change().dropna().std()
    daily_volatility = daily_volatility.sort_values(ascending=False)
    
    annualize_volatility = daily_volatility * np.sqrt(252)
    
    return annualize_volatility

def weekly_str(df, start, end):
    df = df.loc[start:end,].resample('W').last()
    df = df.dropna(axis=1)
    weekly_volatility = df.pct_change().dropna().std()
    weekly_volatility = weekly_volatility.sort_values(ascending=False)
    
    annualize_volatility = weekly_volatility * np.sqrt(52)
    
    return annualize_volatility

def monthly_str(df, start, end):
    df = df.loc[start:end,].resample('M').last()
    df = df.dropna(axis=1)
    monthly_volatility = df.pct_change().dropna().std()
    monthly_volatility = monthly_volatility.sort_values(ascending=False)
    
    annualize_volatility = monthly_volatility * np.sqrt(12)
    
    return monthly_volatility
    
# def quarterly_str(df, start, end):
#     df = df.loc[start:end,]
#     df = df.dropna(axis=1)
#     quarterly_volatility = df.pct_change(63).dropna().std()
#     quarterly_volatility = quarterly_volatility.sort_values(ascending=False)
    
#     return quarterly_volatility

# def halfly_str(df, start, end):
#     df = df.loc[start:end,]
#     df = df.dropna(axis=1)
#     halfly_str = df.pct_change(126).dropna().std()
#     halfly_str = halfly_str.sort_values(ascending=False)
    
#     return halfly_str

# def yearly_str(df, start, end):
#     df = df.loc[start:end,]
#     df = df.dropna(axis=1)
#     yearly_volatility = df.pct_change(252).dropna().std()
#     yearly_volatility = yearly_volatility.sort_values(ascending=False)
    
#     return yearly_volatility

def str_analysis(df, start, end):
    daily_volatility = daily_str(df, start, end)
    weekly_volatility = weekly_str(df, start, end)
    monthly_volatility = monthly_str(df, start, end)
    
    str_df = pd.concat([daily_volatility, weekly_volatility, monthly_volatility], axis=1)
    str_df.columns = ['daily', 'weekly', 'monthly']
    
    return str_df

def str_rank(df):
    df = str_analysis(df, '2011-01-01', '2020-12-31')
    rank_df = df.rank(axis=0, ascending=False, method='min')
    rank_df = rank_df.astype(int)
    
    return rank_df

def low30_str(df):
    df = str_rank(df)
    low30_df = df[(df['monthly'] <= 30) & (df['weekly'] <= 30) & (df['daily'] <= 30)]
    
    return low30_df



In [95]:
str_analysis(equity_universe, '2011-01-01', '2020-12-31')

,daily,weekly,monthly
AMD,0.587842,0.583452,0.163039
PCG,0.571209,0.551477,0.135769
TSLA,0.553647,0.542451,0.179973
FSLR,0.551329,0.541703,0.165738
AAL,0.524601,0.569777,0.140174
...,...,...,...
KO,0.177351,0.182034,0.041365
VZ,0.177314,0.170665,0.045941
PEP,0.176882,0.163367,0.038715
PG,0.175214,0.161953,0.040526


In [97]:
str_rank(equity_universe).head(30)
#low30_str(equity_universe)

,daily,weekly,monthly
AMD,1,1,6
PCG,2,3,12
TSLA,3,5,2
FSLR,4,6,5
AAL,5,2,11
APA,6,7,1
TRGP,7,11,7
FCX,8,8,3
NFLX,9,4,4
VRTX,10,12,30


In [50]:
test = set(daily_str(equity_universe, '2011-01-01', '2011-12-31').rank(ascending=True).iloc[:30].index).intersection(weekly_str(equity_universe, '2011-01-01', '2011-12-31').rank(ascending=True).iloc[:30].index)


In [42]:
weekly_str(equity_universe, '2011-01-01', '2011-12-31').rank(ascending=True).iloc[:30]

NXPI    448.0
URI     447.0
NFLX    446.0
MU      445.0
PHM     444.0
AAL     443.0
SWKS    442.0
STX     441.0
FFIV    440.0
TMUS    439.0
FSLR    438.0
REGN    437.0
NVDA    436.0
MGM     435.0
AMD     434.0
ILMN    433.0
CE      432.0
AKAM    431.0
FTNT    430.0
HAL     429.0
JNPR    428.0
BAC     427.0
DXCM    426.0
VRTX    425.0
MOH     424.0
UAL     423.0
VLO     422.0
C       421.0
MS      420.0
LYB     419.0
dtype: float64

In [38]:
monthly_str(equity_universe, '2011-01-01', '2011-12-31').rank(ascending=False)

STX       1.0
URI       2.0
NFLX      3.0
PHM       4.0
MOH       5.0
        ...  
KMB     444.0
ED      445.0
MCD     446.0
D       447.0
SO      448.0
Length: 448, dtype: float64

In [37]:
quarterly_str(equity_universe, '2011-01-01', '2011-12-31').rank(ascending=False)

URI       1.0
NFLX      2.0
STX       3.0
TMUS      4.0
PHM       5.0
        ...  
MCD     444.0
KMB     445.0
ED      446.0
SO      447.0
D       448.0
Length: 448, dtype: float64

In [25]:
yearly_str(equity_universe, '2011-01-01', '2012-01-04').rank(ascending=False)

CTRA      1.0
FTNT      2.0
MSI       3.0
VFC       4.0
RL        5.0
        ...  
IFF     444.0
AAPL    445.0
DAL     446.0
MHK     447.0
NRG     448.0
Length: 448, dtype: float64